In [1]:
import pandas as pd
import networkx as nx
import math

In [2]:
df = pd.read_csv('Subway map - HK_MTR.csv')
df['GPS'] = df['GPS'].str.split(',')
df['GPS'] = df['GPS'].apply(lambda x: [float(i) for i in x])
df['GPS_lat'] = df['GPS'].apply(lambda x: x[0])
df['GPS_long'] = df['GPS'].apply(lambda x: x[1])
df = df.drop('GPS', axis=1)
df.head()

,Line,Station Name,Count,GPS_lat,GPS_long
0,Island line,Kennedy Town,1,22.281396,114.128882
1,Island line,HKU,1,22.284153,114.135053
2,Island line,Sai Ying Pun,1,22.285722,114.142765
3,Island line,Sheung Wan,1,22.286709,114.151882
4,Island line,Central,2,22.282024,114.158249


In [3]:
def calc_gcd(lat1, lon1, lat2, lon2, earth_radius=6373.0):
    ''' (float, float, float, float, float) -> float
    Calculate the Great Circle Distance between two points
    '''
    lat1, lon1 = math.radians(lat1), math.radians(lon1)
    lat2, lon2 = math.radians(lat2), math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = c * earth_radius

    return distance

In [4]:
mtr = nx.Graph()
for i in range(len(df)-1):
    if df['Line'][i] == df['Line'][i+1]:
        edge = (df['Station Name'][i], df['Station Name'][i+1]) # tuple of two nodes (u, v)
        _u, _v = edge[0], edge[1]
        _weight = calc_gcd(df['GPS_lat'][i], df['GPS_long'][i], df['GPS_lat'][i+1], df['GPS_long'][i+1])
        mtr.add_edge(_u, _v, weight=_weight) 
    

In [5]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(20,20))
# nx.draw_networkx(mtr, pos=nx.spring_layout(mtr), with_labels=True, node_size=100, font_size=10, font_color='r')
# plt.axis('off')
# plt.show()

# Application:

In [6]:
start_station = 'Kowloon'
end_station = 'LOHAS Park'

# Using Dijkstra's algorithm:
print("Shortest path from {} to {} is: ".format(start_station, end_station))
path = nx.shortest_path(mtr, source=start_station, target=end_station, method='dijkstra', weight=None)

print("Path: " + str(path), end="\n\n")
dist_travel = nx.shortest_path_length(mtr, source=start_station, target=end_station)
print("Distance: {:.2f} km".format(dist_travel))

# Print the path:
df_print = pd.DataFrame({'Line': [], 'Station Name': []})

for _stationName in path:
    _df_filtered = df[df['Station Name']==_stationName] 
    if len(_df_filtered) == 1:
        _lineName = "-> " + _df_filtered['Line'].values[0]

    elif len(_df_filtered) > 1:
        __lineName = _df_filtered['Line'].values
        _lineName = "/" 
        for i in __lineName:
            _lineName += i + "/" 

    df_print = pd.concat([df_print, pd.DataFrame({'Line': [_lineName], 'Station Name': [_stationName]})], ignore_index=True)

df_print 

Shortest path from Kowloon to LOHAS Park is: 
Path: ['Kowloon', 'Olympic', 'Nam Cheong', 'Austin', 'East Tsim Sha Tsui', 'Hung Hom', 'Exhibition Centre', 'Admiralty', 'Wan Chai', 'Causeway Bay', 'Tin Hau', 'Fortress Hill', 'North Point', 'Quarry Bay', 'Yau Tong', 'Tiu Keng Leng', 'Tseung Kwan O', 'Hang Hau', 'Po Lam', 'LOHAS Park']

Distance: 19.00 km


,Line,Station Name
0,/Tung Chung Line & Disneyland Resort Line/Airp...,Kowloon
1,-> Tung Chung Line & Disneyland Resort Line,Olympic
2,/Tung Chung Line & Disneyland Resort Line/Tuen...,Nam Cheong
3,-> Tuen Ma Line,Austin
4,-> Tuen Ma Line,East Tsim Sha Tsui
5,/East Rail Line/Tuen Ma Line/,Hung Hom
6,-> East Rail Line,Exhibition Centre
7,/Island line/Tsuen Wan Line/South Island Line/...,Admiralty
8,-> Island line,Wan Chai
9,-> Island line,Causeway Bay
